In [3]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.vectorstores import Chroma

import sys
from pathlib import Path

sys.path.append(str(Path.cwd().parents[1])) #nopep8

from langchain_community.document_loaders import DirectoryLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.llms import Ollama
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage

from utilities.project_paths import sample_document_dirpath
from utilities.ollama import verify_ollama_model_present


MODEL_NAME = "openchat"
# https://ollama.com/library/openchat

generator_llm = Ollama(model=MODEL_NAME)
embeddings = OllamaEmbeddings(model=MODEL_NAME)

# make sure that the ollama is available to use locally (might need to ollama pull)
verify_ollama_model_present("openchat")

validating model exists...


In [34]:
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import UnstructuredMarkdownLoader

from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

# load the document and split it into chunks
document_path = sample_document_dirpath / 'cross_service.md'
loader = UnstructuredMarkdownLoader(str(document_path))
documents = loader.load()

# split it into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(documents)

# load it into Chroma
db = Chroma.from_documents(docs, embeddings, persist_directory="./chroma_db")

In [35]:
query = "What AWS services might I use for creating a websocket chat?"
docs = db.similarity_search(query)

# print results
# print(docs[0].page_content)
docs

[Document(page_content='Aurora\nAmazon SES\n\nAmazon Aurora Serverless REST API lending library example\nShows how to use the Amazon Relational Database Service (Amazon RDS) API and \nAWS Chalice to create a REST API backed by an Amazon Aurora database. The web \nservice is fully serverless and represents a simple lending library where patrons \ncan borrow and return books.\n\nAmazon API Gateway\nAWS Lambda\nAmazon RDS\nAWS Secrets Manager\n\nTrack work items in a DynamoDB table\nShows how to create a REST service that lets you store work items in an \nAmazon DynamoDB table and use Amazon Simple Email Service (Amazon SES) \nto send email reports of work items.\n\nAmazon DynamoDB\nAmazon SES\n\nAnalyzing photos using Amazon Rekognition\nShows you how to create a web application that lets you upload photos to an \nAmazon Simple Storage Service (Amazon S3) bucket, use Amazon Rekognition to analyze \nand label the photos, and use Amazon Simple Email Service (Amazon SES) to send \nemail rep